# 搭建并使用向量数据库

## 前序数据预处理配置

批量处理文件夹中所有文件

In [1]:
import os
from dotenv import load_dotenv, find_dotenv 
# pip install python-dotenv

_ = load_dotenv(find_dotenv())


# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../../data_base/knowledge_path'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

['../../data_base/knowledge_path/pumkin_book/pumpkin_book.pdf']


In [2]:
from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

In [3]:
# # 下载文件并存储到text
# # 未作数据清洗
# texts = []

# for loader in loaders:
#     texts.extend(loader.load())

# texts[2]

In [4]:
# 下载文件并存储到text
# 作数据清洗
from langchain.document_loaders.pdf import PyMuPDFLoader
import re
# 下载文件并存储到text
texts = []
for loader in loaders:
    texts.extend(loader.load())   

for text in texts:
    pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
    text.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), text.page_content)
    text.page_content = text.page_content.replace('•', '')
    text.page_content = text.page_content.replace(' ', '')

In [5]:
texts[0].page_content

'\x01本\x03:1.9.9\n发布日期:2023.03\n南⽠书\nPUMPKINBOOKDatawhale'

In [ ]:
# # 下载文件并存储到text
# # 作数据清洗
# from langchain.document_loaders.pdf import PyMuPDFLoader
# import re
# # 下载文件并存储到text
# texts = []
# for loader in loaders:
#     texts.extend(loader.load())   

# for text in texts:
#     text.page_content = re.sub(r'\s*\n\s*', ' ', text.page_content)
#     text.page_content = re.sub(r'[\s•]', '', text.page_content)

载入后的变量类型为`langchain_core.documents.base.Document`, 文档变量类型同样包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [6]:
text = texts[100]
print(f"每一个元素的类型：{type(text)}.", 
    # f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
查看该文档的内容:
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←
当某一个类别j的基分类器的结果之和，大于所有结果之和的12，则选择该类别j为最终结果。8.4.5
式(8.25)的解释
H(x)=cargmaxj
∑Ti=1hji(x)
相比于其他类别，该类别j的基分类器的结果之和最大，则选择类别j为最终结果。8.4.6
式(8.26)的解释
H(x)=cargmaxj
∑Ti=1wihji(x)
相比于其他类别，该类别j的基分类器的结果之和最大，则选择类别j为最终结果，与式(8.25)不同的
是，该式在基分类器前面乘上一个权重系数，该系数大于等于0，且T个权重之和为1。8.4.7
元学习器(meta-learner)的解释
书中第183页最后一行提到了元学习器(meta-learner)，简单解释一下，因为理解meta的含义有时
对于理解论文中的核心思想很有帮助。
元(meta)，非常抽象，例如此处的含义，即次级学习器，或者说基于学习器结果的学习器；另外还有
元语言，就是描述计算机语言的语言，还有元数学，研究数学的数学等等；
另外，论文中经常出现的还有meta-strategy，即元策略或元方法，比如说你的研究问题是多分类问题，
那么你提出了一种方法，例如对输入特征进行变换（或对输出类别做某种变换），然后再基于普通的多分
类方法进行预测，这时你的方法可以看成是一种通用的框架，它虽然针对多分类问题开发，但它需要某个
具体多分类方法配合才能实现，那么这样的方法是一种更高层级的方法，可以称为是一种meta-strategy。8.4.8Stacking算法的解释
该算法其实非常简单，对于数据集，试想你现在有了个基分类器预测结果，也就是说数据集中的每个
样本均有个预测结果，那么怎么结合这个预测结果呢？
本节名为“结合策略”，告诉你各种结合方法，但其实最简单的方法就是基于这个预测结果再进行一
次学习，即针对每个样本，将这个预测结果作为输入特征，类别仍为原来的类别，既然无法抉择如何将这
些结果进行结合，那么就“学习”一下吧。“西瓜书”图8.9伪代码第9行中将第个样本进行变换，特征为个基学习器的输出，类别标记仍为原
来的，将所

In [7]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [9]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(text.page_content[0:1000])

In [15]:
split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：627


In [16]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：252347


In [17]:
split_docs[2].page_content

'有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力(zhi)争(neng)以本科数学基础的视角进行讲解，所以超纲的数学知识\n我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们GitHub的\nIssues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块\n提交你希望补充的公式编号或者勘误信息，我们通常会在24小时以内给您回复，超过24小时未回复的\n话可以微信联系我们（微信号：at-Sm1les）；\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1版）\n最新版PDF获取地址：https://github.com/datawhalechina/pumpkin-book/releases\n编委会'

## 构建Milvus向量库

### 构建Chroma向量库-embedding - zhipu

Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

In [18]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_milvus import Milvus

# 文本嵌入
embed = ZhipuAIEmbeddings(model="embedding-2",api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 向量库创建
connection_args = {
    "host": "129.201.70.31",
    "port": "19530",
}
vectordb = Milvus.from_documents(
    documents=split_docs,
    embedding=embed,
    collection_name="book2",
    drop_old=False,
    connection_args=connection_args,
)


# print(vectordb._collection.count())

# 检索
question = "图像识别"
docs = vectordb.similarity_search(question,k=3)
print(len(docs))
print(docs[0].page_content)

APIRequestFailedError: Error code: 400, with error text {"error":{"code":"1214","message":"input数组最大不得超过64条"}}

In [ ]:
# 使用 OpenAI Embedding
# from langchain.embeddings.openai import OpenAIEmbeddings
# 使用百度千帆 Embedding
# from langchain.embeddings.baidu_qianfan_endpoint import QianfanEmbeddingsEndpoint
# 使用我们自己封装的智谱 Embedding，需要将封装代码下载到本地使用
from zhipuai_embedding import ZhipuAIEmbeddings 

# 定义 Embeddings
# embedding = OpenAIEmbeddings() 
embedding = ZhipuAIEmbeddings()
# embedding = QianfanEmbeddingsEndpoint()

# 定义持久化路径
persist_directory = '../../data_base/vector_db/chroma'

In [ ]:
# !rm -rf '../../data_base/vector_db/chroma'  # 删除旧的数据库文件（如果文件夹中有文件的话），windows电脑请手动删除

**安装chromadb， 为了避免版本兼容问题，注意安装指定版本**




In [ ]:
# !pip install chroma-hnswlib==0.7.1 chromadb==0.4.3

In [ ]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents=split_docs[:20], # 为了速度，只选择前 20 个切分的 doc 进行生成
    # documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

在此之后，我们要确保通过运行 vectordb.persist 来持久化向量数据库，以便我们在未来的课程中使用。

让我们保存它，以便以后使用！

In [ ]:
vectordb.persist()
# vectordb.PersistentClient(persist_directory)

In [ ]:
# import os
# from langchain.vectorstores.chroma import Chroma
# from zhipuai_embedding import ZhipuAIEmbeddings   # 假设这是正确的导入路径

# # 定义持久化目录
# persist_directory = '../../data_base/vector_db/chroma'

# # 创建嵌入模型
# embedding = ZhipuAIEmbeddings()

# try:
#     # 初始化 Chroma 向量数据库
#     vectordb = Chroma.from_documents(
#         documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
#         embedding=embedding,
#         persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
#     )
    
#     # 持久化向量数据库
#     vectordb.persist()
#     print("向量数据库已成功持久化到磁盘。")
# except Exception as e:
#     print(f"持久化过程中发生错误: {e}")

In [ ]:
# 汇总

import os
from langchain.vectorstores.chroma import Chroma
from pydantic import BaseModel  # 直接从 pydantic 导入
from zhipuai_embedding import ZhipuAIEmbeddings  # 假设这是正确的导入路径

# 定义持久化目录
persist_directory = '../../data_base/vector_db/chroma'

# 创建嵌入模型
embedding = ZhipuAIEmbeddings()

try:
    # 初始化 Chroma 向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=embedding,
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

### 构建Chroma向量库-embedding - hugging face

In [ ]:
# 使用 HuggingFaceEmbeddings API， 免费
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.vectorstores.chroma import Chroma
# from langchain_community.vectorstores import Chroma

In [ ]:
import os
from langchain.vectorstores.chroma import Chroma
from pydantic import BaseModel  # 直接从 pydantic 导入
from langchain_community.embeddings import HuggingFaceEmbeddings  

# 定义持久化目录
persist_directory = '../../data_base/vector_db/chroma'

# 创建嵌入模型
h_embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")


try:
    # 初始化 Chroma 向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=h_embedding,
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

### 构建Chroma向量库-embedding - ollama

In [ ]:
# from langchain.embeddings import OllamaEmbeddings

# # 初始化Ollama嵌入模型
# # 假定Ollama服务已经在本地运行
# oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="qwen2:1.5b")

# # 假设`split_docs`包含了我们之前分割的文档片段
# # 示例数据
# split_docs = ["这是一个例子文档片段。", "这是另一个例子文档片段。"]

# # 为文档片段生成向量
# vectors = [oembed.embed_query(embedding_text) for embedding_text in split_docs]

# # 打印出第一个文档片段的向量，以验证向量生成是否成功
# print(vectors[0][:10])

In [ ]:
import os
from langchain.vectorstores.chroma import Chroma
from pydantic import BaseModel  # 直接从 pydantic 导入
from langchain.embeddings import OllamaEmbeddings

# 定义持久化目录
persist_directory = '../../data_base/vector_db/chroma'

# 创建嵌入模型
# 初始化Ollama嵌入模型
# 假定Ollama服务已经在本地运行
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

try:
    # 初始化 Chroma 向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=oembed,
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

## 三、向量检索
### 3.1 相似度检索
Chroma的相似度搜索使用的是余弦距离，即：
$$
similarity = cos(A, B) = \frac{A \cdot B}{\parallel A \parallel \parallel B \parallel} = \frac{\sum_1^n a_i b_i}{\sqrt{\sum_1^n a_i^2}\sqrt{\sum_1^n b_i^2}}
$$
其中$a_i$、$b_i$分别是向量$A$、$B$的分量。

当你需要数据库返回严谨的按余弦相似度排序的结果时可以使用`similarity_search`函数。

In [ ]:
question="什么是南瓜书"

In [ ]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

In [ ]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

### 3.2 MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [ ]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [ ]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")